In [ ]:
import pandas as pd
import numpy as np

# Read the gzipped CSV directly
df = pd.read_csv('demographics_all_v3.csv.gz', compression='gzip')


# Display the first few rows
print(df.head())

In [ ]:
#1584 unique patients
len(df[df["ad_only"]==1]['subject_id'].unique())

In [ ]:
# Filter for ad_only == 1
ad_only_df = df[df["ad_only"] == 1]

# Group by subject_id and count the number of rows for each subject
subject_counts = ad_only_df.groupby('subject_id')["hadm_id"].count()

import matplotlib.pyplot as plt
# Display the distribution of row counts
plt.hist(subject_counts,bins=100)
plt.title("treatment patient entries")
plt.show()


In [ ]:
# Assuming 'subject_counts' is the pandas Series you created earlier
cutoff = np.percentile(subject_counts, 95)
print(f"The 95th percentile cutoff is: {cutoff}")

In [ ]:
subjects_multiple_rows = subject_counts[subject_counts > 1].index

# Print the first 5 such subjects
for subject_id in subjects_multiple_rows[:5]:
    print(f"Rows for subject_id: {subject_id}")
    print(ad_only_df[ad_only_df['subject_id'] == subject_id])
    print("-" * 20)
    break


In [ ]:
df['race_group'].value_counts()

In [ ]:
df['admission_type'].value_counts()

In [ ]:
df.columns

In [ ]:
# Filter for ad_only == 1
control_df = df[df["ad_only"] == 0]

# Group by subject_id and count the number of rows for each subject
subject_counts = control_df.groupby('subject_id')["hadm_id"].count()

import matplotlib.pyplot as plt
# Display the distribution of row counts
plt.hist(subject_counts,bins=100)
plt.title("control patient entries")
plt.show()


In [ ]:
# Assuming 'subject_counts' is the pandas Series you created earlier
cutoff = np.percentile(subject_counts, 95)
print(f"The 95th percentile cutoff is: {cutoff}")

In [201]:
treatment_col = "ad_only"
covariates = ["gender", "race_group", "age",'admission_type',"marital_status", 'language_group', "insurance"]

## Clean dataframe so unique treatment group id

In [ ]:
# Create a copy to avoid modifying the original DataFrame
new_df = df.copy()

# Convert admission date to datetime for proper comparison
new_df['admit_date'] = pd.to_datetime(new_df[['admityear', 'admitmonth', 'admitday']].rename(columns={
    'admityear': 'year',
    'admitmonth': 'month',
    'admitday': 'day'
}))

# Function to process each group
def process_group(group):
    if group['ad_only'].iloc[0] == 1:
        # Find row with earliest admission date
        min_idx = group['admit_date'].idxmin()
        # Keep only this row
        group = group.loc[[min_idx]]
    return group

# Group by 'subject_id' and apply the function
new_df = new_df.groupby('subject_id', group_keys=False).apply(process_group)

# Drop the temporary date column if needed
new_df = new_df.drop(columns=['admit_date'])

# Reset index
new_df = new_df.reset_index(drop=True)

# Display the result
print(new_df.head())

In [203]:
# prompt: for ad_only=0 rows, remove all rows if it's subject_id is in ad_only=1 dataframe

ad_only_df=new_df[new_df["ad_only"]==1]
control_df=new_df[new_df["ad_only"]==0]

# Get subject IDs from the ad_only=1 DataFrame
ad_only_subjects = ad_only_df['subject_id'].unique()

# Filter out rows from the control DataFrame where subject_id is in ad_only_subjects
control_df = control_df[~control_df['subject_id'].isin(ad_only_subjects)]

new_df=pd.concat([ad_only_df, control_df])

In [ ]:
# prompt: check if there's duplicative subject_id

import pandas as pd

# Assuming 'df' is your DataFrame (as defined in the provided code)
duplicate_subjects = ad_only_df[ad_only_df.duplicated(subset=['subject_id'], keep=False)]

if not duplicate_subjects.empty:
  print("Duplicate subject_ids found:")
  print(duplicate_subjects[['subject_id']])
else:
  print("No duplicate subject_ids found.")


In [ ]:
len(new_df[new_df["ad_only"]==1])

# covariate balance matching - DAME-FLAME


When to use:

You want to retain all data points but ensure covariates (e.g., age, education) have similar distributions across outcomes.

Ideal for predictive modeling (e.g., training a classifier to predict Alzheimer’s).

Pros:

- Retains all data.

- Reduces bias without discarding samples.

Cons:

- Does not guarantee exact matches for causal inference.

### Intro of package
Flame:
Begin with iteration 1 by matching units exactly on the entire set of covariates. Then ieratively drop least important covariates to match until the remaining set of covariates cannot be used to accurately predict the outcome on trianing set

Dame:(Dynamic Almost-Matching Exact)
switch to second good combination of features instead of eliminate one at a time

Frame-Dame:
Frame until a point then Dame

| Feature               | dame-flame                                                                                 | pymatch (Propensity Score)                                     |
|-----------------------|--------------------------------------------------------------------------------------------|-----------------------------------------------------------------|
| **Matching Approach** | Almost exact matching on subsets of covariates using ML-guided iterative approach         | Propensity scores (logistic regression/CatBoost)               |
| **Categorical Handling** | No encoding needed; matches raw discrete covariates directly                           | Requires one-hot/dummy encoding                                |
| **Interpretability**  | Can match explicit covariate subsets (e.g., Age=65, Sex=Female)                          | Opaque propensity scores (0.23 vs. 0.25)                        |
| **Bias Reduction**    | Prioritizes matches on outcome-predictive covariates via holdout training                 | Relies on correct propensity model specification                |
| **Scalability**       | Optimized for high-cardinality categorical data                                           | Better for large datasets with continuous covariates           |


In [ ]:
from dame_flame.matching import DAME

In [ ]:
# prompt: check null value of covariates in df

# Check for null values in the specified covariates
null_counts = df[covariates].isnull().sum()
null_counts


### Only do matching on non-null features

In [ ]:
# Initialize DAME model
covariates_selected = ["gender", "race", "age", 'admission_type']

# For multiple categorical columns:
cat_cols = ['gender', 'race','admission_type']

df_selected=df[covariates_selected]
df_selected["treated"]=df[treatment_col]
df_selected['outcome'] = 0 # Add dummy outcome (all zeros), cuz the FLAME-DAME requires outcome
df_selected[cat_cols] = df_selected[cat_cols].apply(lambda x: pd.factorize(x)[0]) #convert features to integers

df_selected.head()

In [ ]:
# After running your model specification:
model = DAME(
    repeats=False,          # 1:1 matching (no reuse of controls)
    early_stop_pe=0,        # Disable outcome-based stopping
)

model.fit(df_selected, treatment_column_name='treated',outcome_column_name='outcome')

In [ ]:
matched_data = model.predict(df_selected)
matched_data

In [ ]:
# prompt: get the rows of df_selected with the same row index as in all_matched_data

# Get the indices of the matched rows
matched_indices = matched_data.index

# Select rows from df_selected using these indices
df_matched_selected = df.loc[matched_indices]

# Now df_matched_selected contains the rows from df_selected that were matched
# and have the same row index as in all_matched_data.
df_matched_selected


In [ ]:
df_matched_selected["adrd"].value_counts()

In [ ]:
df["adrd"].value_counts()

In [ ]:
# prompt: plot covariate distribution between adrd groups for each covariate in covariates list, do subplots and make the x labels vertical

import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'df_matched_selected' and 'covariates' are defined as in the previous code

fig, axes = plt.subplots(len(covariates), 1, figsize=(8, 6 * len(covariates)))

for i, covariate in enumerate(covariates):
    sns.histplot(data=df_matched_selected, x=covariate, hue="adrd", kde=True, ax=axes[i], element="step")
    axes[i].set_xlabel(covariate, fontsize=12)
    axes[i].set_ylabel("Density", fontsize=12)
    axes[i].tick_params(axis='x', rotation=90) # Rotate x-axis labels vertically

plt.tight_layout()
plt.savefig('adrd_distr_covariate_distribution.png')
plt.show()

In [ ]:
df_matched_selected.to_csv('matched_adrd_distr.csv')

# 1:1 matching

AD_only: 1584 VS 1584

When to use:

You need causal comparability (e.g., estimating treatment effects or isolating outcome-specific associations).

Critical for observational studies where unmeasured confounding is a risk.

Methods:

Exact Matching (for categorical covariates):

Pros:

- Creates directly comparable groups.

- Mimics randomized controlled trials.

Cons:

- Discards unmatched data (sample loss).

- Computationally intensive for large datasets.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(len(covariates), 1, figsize=(8, 6 * len(covariates)))

for i, covariate in enumerate(covariates):
    sns.histplot(data=new_df, x=covariate, hue=treatment_col, kde=True, ax=axes[i], element="step")
    axes[i].set_xlabel(covariate, fontsize=12)
    axes[i].set_ylabel("Density", fontsize=12)
    axes[i].tick_params(axis='x', rotation=90) # Rotate x-axis labels vertically

plt.tight_layout()
plt.savefig(f'{treatment_col}_before_distribution.png')
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

df2=new_df[covariates]
df2[treatment_col]=new_df[treatment_col]

# Encode categorical variables
df2 = pd.get_dummies(df2, columns= ["gender", "race_group",'admission_type',"marital_status", 'language_group', "insurance"], drop_first=True)

# Calculate propensity scores
X = df2.drop(columns=[treatment_col])
y = df2[treatment_col]

ps_model = LogisticRegression(max_iter=1000, random_state=42)
ps_model.fit(X, y)
df2["propensity_score"] = ps_model.predict_proba(X)[:, 1]
df2["subject_id"]=new_df["subject_id"]

# 1:1 matching with caliper
treated = df2[df2[treatment_col] == 1]
controls = df2[df2[treatment_col] == 0]

In [208]:
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

def ultra_fast_1to1_matching_with_exclusion(treated, controls, caliper=0.2):
    """
    Optimized 1:1 matching with no control reuse and exclusion of all rows
    with the same subject_id when a control is matched.
    """
    # Convert to numpy arrays for speed
    treated_ps = treated["propensity_score"].values.reshape(-1, 1)
    control_ps = controls["propensity_score"].values.reshape(-1, 1)
    control_indices = controls.index.values
    control_subject_ids = controls["subject_id"].values  # Track subject_ids

    # Standardize
    scaler = StandardScaler()
    treated_ps_std = scaler.fit_transform(treated_ps)
    control_ps_std = scaler.transform(control_ps)

    # Build BallTree for fast radius searches
    tree = BallTree(control_ps_std)

    # Track used controls with boolean array (faster than set)
    used_controls = np.zeros(len(controls), dtype=bool)
    matches = []

    # Process in random order to avoid quality bias
    rng = np.random.default_rng(42)
    treated_order = rng.permutation(len(treated))

    for i in treated_order:
        ps = treated_ps_std[i]
        # Find all potential matches within caliper (radius search)
        match_indices = tree.query_radius([ps], r=caliper)[0]

        # Mask for unused controls
        available_mask = ~used_controls[match_indices]
        available_indices = match_indices[available_mask]

        if len(available_indices) > 0:
            # Find closest available control (using numpy argmin)
            distances = np.abs(control_ps_std[available_indices].flatten() - ps[0])
            best_idx = available_indices[np.argmin(distances)]

            # Mark ALL rows with this subject_id as used
            matched_subject_id = control_subject_ids[best_idx]
            used_controls[control_subject_ids == matched_subject_id] = True

            matches.append({
                'treated_idx': treated.index[i],
                'control_idx': control_indices[best_idx]
            })

    return matches

# Usage
matches = ultra_fast_1to1_matching_with_exclusion(treated, controls, caliper=0.2)

# Create matched dataset (vectorized operations)
treated_matched = treated.loc[[m['treated_idx'] for m in matches]]
controls_matched = controls.loc[[m['control_idx'] for m in matches]]
matched_df = pd.concat([treated_matched, controls_matched])

# Add pair IDs in one operation
matched_df['pair_id'] = np.repeat(np.arange(len(matches)), 2)

In [ ]:
len(matched_df)

In [ ]:
# Check balance
def check_balance(df, covariates):
    balance = {}
    for var in covariates:
        if var in df.columns:
            t_mean = df[df[treatment_col] == 1][var].mean()
            c_mean = df[df[treatment_col] == 0][var].mean()
            pooled_std = np.sqrt((df[df[treatment_col] == 1][var].var() +
                                df[df[treatment_col] == 0][var].var()) / 2)
            balance[var] = abs((t_mean - c_mean) / pooled_std)
    return balance

balance_report = check_balance(matched_df, covariates)
print("Balance Report (SMD < 0.1 is good):")
for var, smd in balance_report.items():
    print(f"{var}: {smd:.3f}")


print(f"\nMatched {len(treated_matched)} treated to {len(controls_matched)} controls")

In [211]:
# prompt: get the rows of df_selected with the same row index as in all_matched_data

# Get the indices of the matched rows
matched_indices = matched_df.index

# Select rows from df_selected using these indices
df_matched_selected = new_df.loc[matched_indices]

# Now df_matched_selected contains the rows from df_selected that were matched
# and have the same row index as in all_matched_data.
df_matched_selected
df_matched_selected.to_csv(f'matched_{treatment_col}.csv')

In [ ]:
print(df_matched_selected.head())
print(len(df_matched_selected))

In [ ]:
# prompt: check if there's duplicative value for subject_id

# Assuming 'df' is your DataFrame as defined in the provided code.
duplicate_subjects = df_matched_selected[df_matched_selected.duplicated(subset=['subject_id'], keep=False)]

if not duplicate_subjects.empty:
    print("Duplicate subject IDs found:")
    print(duplicate_subjects[['subject_id']])
else:
    print("No duplicate subject IDs found.")


In [ ]:
# prompt: check if df_matched_selected has duplicative index, if yes return index

# Check for duplicate indices
duplicate_indices = df_matched_selected.index[df_matched_selected.index.duplicated(keep=False)]

if not duplicate_indices.empty:
  print("Duplicated indices found:")
  print(duplicate_indices)
else:
  print("No duplicate indices found.")


In [ ]:
fig, axes = plt.subplots(len(covariates), 1, figsize=(8, 6 * len(covariates)))

for i, covariate in enumerate(covariates):
    sns.histplot(data=df_matched_selected, x=covariate, hue=treatment_col, kde=True, ax=axes[i], element="step")
    axes[i].set_xlabel(covariate, fontsize=12)
    axes[i].set_ylabel("Density", fontsize=12)
    axes[i].tick_params(axis='x', rotation=90) # Rotate x-axis labels vertically

plt.tight_layout()
plt.savefig(f'{treatment_col}_after_distribution.png')
plt.show()